# Simulation analysis
A subset of the analysis performed for the corresponding manuscript is provided below.

In [ ]:
import os
import errno
import boto3
import shutil
import pandas
import numpy

from itertools import cycle
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import interp1d, spline
from mpl_toolkits.axes_grid1 import make_axes_locatable
# from matplotlib.ticker import FormatStrFormatter

%matplotlib inline

In [ ]:
input_dir = '../data/'
output_dir = '../results/'

## Functions

In [ ]:
def assert_dir_exists(path):
    """
    Checks if directory tree in path exists. If not it created them.
    :param path: the path to check if it exists
    """
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def download_dir(client, bucket, path, target):
    """
    Downloads recursively the given S3 path to the target directory.
    :param client: S3 client to use.
    :param bucket: the name of the bucket to download from
    :param path: The S3 directory to download.
    :param target: the local directory to download the files to.
    """

    # Handle missing / at end of prefix
    if not path.endswith('/'):
        path += '/'

    paginator = client.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket, Prefix=path):
        # Download each file individually
        for key in result['Contents']:
            # Calculate relative path
            rel_path = key['Key'][len(path):]
            # Skip paths ending in /
            if not key['Key'].endswith('/'):
                local_file_path = os.path.join(target, rel_path)
                # Make sure directories exist
                local_file_dir = os.path.dirname(local_file_path)
                assert_dir_exists(local_file_dir)
                client.download_file(bucket, key['Key'], local_file_path)
                
                
def fix_header_for_pandas(directory, file_prefix, suffix):
    all_files = os.listdir(directory)
    for each_file in all_files:
        if (each_file.startswith(file_prefix)) and (each_file.endswith(suffix)):
            full_file = os.path.join(directory, each_file)
            from_file = open(full_file) 
            line = from_file.readline()
            # Check is a header is provided
            if line.startswith('HEADER'):
                # Truncate header line to just have names of columns
                revised_line = (',').join(line.split(',')[1:])
                revised_file = open(os.path.join(directory, 'pdH_{}'.format(each_file)),mode="w")
                revised_file.write(revised_line)
                shutil.copyfileobj(from_file, revised_file)
                os.remove(full_file)
                
def makeLongId(cur_id, sim=0, seed=1000):
    long_id = '{:04d}{:04d}{}'.format(sim, seed, cur_id)
    return long_id

def makeLongIdByRow(row, seed=1000):
    if '.' in row['Depth']:
        cur_depth = str(row['Depth'].replace('.', '00'))
    else:
        cur_depth = '{:02d}'.format(int(row['Depth']))
    long_id = '{:04d}{}{}{}'.format(seed, cur_depth, int(row['EventID']), int(row['TrackID']))
    return long_id
    
def readGamosOutputFile(directory, file_prefix, suffix, headers=[], chunksize=10**6):
    list_ = []
    all_files = os.listdir(directory)
    for each_file in all_files:
        if (each_file.startswith(file_prefix)) and (each_file.endswith(suffix)):
            full_file = os.path.join(directory, each_file)
            if headers==[]:
                for chunk in pandas.read_csv(full_file, delimiter=',', header=0, iterator=True, chunksize=chunksize):
                    list_.append(chunk)
            else:
                # Use custom headers
                for chunk in pandas.read_csv(full_file, delimiter=',', names=headers, iterator=True, chunksize=chunksize):
                    list_.append(chunk)
    df = pandas.concat(list_)
            
    return df

def readMultiGamosOutputFiles(directory, file_prefix, headers=[], chunksize=10**6):
    df = pandas.DataFrame()
    dirs = []
    all_contents = os.listdir(directory)
    for each_entry in all_contents:
        if os.path.isdir(os.path.join(directory, each_entry)):
            dirs.append(each_entry)
    if len(dirs)>0:
        for each_dir in dirs:
            rand_seed = each_dir.split('_')[-1]
            depth = each_dir.split('_')[-2]
            suffix = '{}_{}'.format(depth, rand_seed)
#             print(suffix)
            tmp_df = readGamosOutputFile(os.path.join(directory, each_dir), file_prefix, suffix)
            if tmp_df.empty:
#                 print('\tempty')
                df=df
            else:
                tmp_df['Depth'] = str(depth)
                tmp_df['Seed'] = int(rand_seed)
                tmp_df['LongID'] = tmp_df.apply(makeLongIdByRow, args=[int(rand_seed)], axis=1)
                df = df.append(tmp_df)
#                 print(len(df))
    else:
        df = readGamosOutputFile(directory, suffix, headers, chunksize)
    return df

def readGamosFluenceFile(file, readInError=False, divide_by_sum=False):
    new_file = "{}.new".format(file)
    with open(file, "r") as f:
        file_contents = f.read()
        new_index = 0
        i = 0
        while i<5:
            new_index = file_contents.find("\n", new_index+1)
            i +=1
    with open(new_file, "w") as f:
        f.write(file_contents[new_index:])
    if readInError:
        df = pandas.read_csv(new_file, delimiter=',', names=['voxel','fluence', 'error(rel)', 'sumV2'])   
    else:
        df = pandas.read_csv(new_file, delimiter=',', names=['voxel','fluence'])
    if divide_by_sum:
        #print(df.tail(1)['voxel'].values[0].split(' ')[1])
        total_sum = float(df.tail(1)['voxel'].values[0].split(' ')[1])
        df = df[:-1]
        new_fluence_values = df['fluence']/total_sum
        df['fluence'] = new_fluence_values
    else:
        df = df[:-1]
    return df

def readGamosDoseFile(file):
    new_file = "{}.new".format(file)
    with open(file, "r") as f:
        file_contents = f.read()
        new_index = 0
        i = 0
        while i<5:
            new_index = file_contents.find("\n", new_index+1)
            i +=1
    with open(new_file, "w") as f:
        f.write(file_contents[new_index:])
    df = pandas.read_csv(new_file, delimiter=',', names=['voxel','dose', 'rel_error', 'sumV2'])
    df = df[:-1]
    return df

def moving_average(a, n=10) :
    ret = numpy.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def integrate_spectrum(amplitudes, wavelengths, a, b):
    """Integrate a spectrum from a to b
    :param amplitudes: An array of length N containing amplitudes for a given spectrum
    :param wavelengths: An array of length N containing wavelengths whigh correspond 
        to the amplitudes for a given spectrum
    :param a: The starting wavelength for integration of the spectrum
    :param b: The ending wavelength for integration of the spectrum
    :returns: The area under the spectrum using composite trapezoidal rule
    """
    #Find subset of array where spectrum is between a and b
    #num_measurements = amplitudes.shape[0][0]/amplitudes.shape[0][1]
    spec_roi = amplitudes[numpy.where((wavelengths>=a) & (wavelengths<=b))]
    wv_roi = wavelengths[numpy.where((wavelengths>=a) & (wavelengths<=b))]
    dx = (wv_roi[1] - wv_roi[0])
    area = numpy.trapz(spec_roi, wv_roi, dx=dx)
        
    return area

## Download simulation output from S3

In [ ]:
s3_client = boto3.client('s3')

In [ ]:
s3_bucket_6mv_name = 'gamos-ptg4-6mv-nofluence'
root_path_name = '6MV_NF'
s3_paths = []
# S3 folders were created programatically to have a structured name
for depth in [0,1,2,3,5,7]:
    for i in range(0,10):
        random_seed = 1000+i
        cur_path_name = '{}_{}_{}'.format(root_path_name, depth, random_seed)
        s3_paths.append(cur_path_name)

In [ ]:
# Using list of S3 folders in bucket, copy each to local system
for each_dir in s3_paths:
    new_dir = os.path.join(output_dir, 'simulations', root_path_name , each_dir)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    download_dir(s3_client, s3_bucket_6mv_name ,  each_dir, new_dir)

In [ ]:
# Modify all the detector files with headers to make it easier for pandas to parse
for each_dir in s3_paths:
    sim_dir = os.path.join(output_dir, 'simulations', root_path_name, each_dir)
    fix_header_for_pandas(sim_dir, 'Detect_', each_dir)

## Fluorescence at surface

In [ ]:
fl_at_surface = readMultiGamosOutputFiles(os.path.join(output_dir, 'simulations', root_path_name),
                                          'pdH_Detect_fl_at_surface')

In [ ]:
surface_fl = {}
total_str = ''
for each_depth in ['0', '1', '2', '3', '5', '7']:
    surface_fl[depth] = plt.hist2d(fl_at_surface[fl_at_surface['Depth']==each_depth]['FinalPosX'],
               fl_at_surface[fl_at_surface['Depth']==each_depth]['FinalPosY'],
               numpy.arange(-30,30))
    center = surface_fl[depth][0][29:32, 29:32]
    max_center = numpy.nanmax(center)
    cur_avg = numpy.nanmean(surface_fl[depth][0][surface_fl[depth][0]>(0.5*max_center)])
    fwhm_count = numpy.sum(surface_fl[depth][0]>(0.5*max_center))
    cur_fwhm = 2*numpy.sqrt(fwhm_count/numpy.pi)
    new_line = '{}mm:\tFWHM:{:1.2f}mm,\tMax:{:1.0f},\tMean:{:1.1f}'.format(each_depth, cur_fwhm, max_center, cur_avg)
    print(new_line)
    total_str=total_str+new_line+'\n'
with open(os.path.join(output_dir, '6MV_fluorescence_FWHM.txt'), "w") as text_file:
    print(total_str, file=text_file)

In [ ]:
plt.figure(figsize=[8,8])
hist2d_6MV_7 = plt.hist2d(fl_at_surface[fl_at_surface['Depth']=='2']['FinalPosX'],
               fl_at_surface[fl_at_surface['Depth']=='2']['FinalPosY'],
               numpy.arange(-30,30))
plt.colorbar()
plt.show()

## Fluoresence starting in tumor

In [ ]:
fl_starting_in_tumor = readMultiGamosOutputFiles(os.path.join(output_dir, 'simulations', root_path_name),
                                          'pdH_Detect_fl_start_tumor')

## Fluoresence exiting tumor

In [ ]:
fl_exiting_tumor = readMultiGamosOutputFiles(os.path.join(output_dir, 'simulations', root_path_name),
                                          'pdH_Detect_fl_exit_tumor')

## Fluoresence attenuation

In [ ]:
# fl_starting_in_tumor[fl_starting_in_tumor['Depth']=='7']['WavelengthEnergy*nm']
# fl_exiting_tumor[fl_exiting_tumor['Depth']=='7']['WavelengthEnergy*nm']
# fl_at_surface[fl_at_surface['Depth']=='7']['WavelengthEnergy*nm']
total_str = ''
for depth in ['0','1','2','3','5','7']:
    starting_fl_count = len(fl_starting_in_tumor[fl_starting_in_tumor['Depth']==depth]['WavelengthEnergy*nm'])
    exiting_fl_count = len(fl_exiting_tumor[fl_exiting_tumor['Depth']==depth]['WavelengthEnergy*nm'])
    surface_fl_count = len(fl_at_surface[fl_at_surface['Depth']==depth]['WavelengthEnergy*nm'])
    percent_exiting = exiting_fl_count/starting_fl_count
    percent_surface = surface_fl_count/starting_fl_count
    new_line = '{}mm:\t(Starting N:{}),\tExiting: {:1.2f}%,\tSurface: {:1.2f}% ({})'.format(depth, starting_fl_count, percent_exiting*100, percent_surface*100, surface_fl_count)
    print(new_line)
    total_str=total_str+new_line+'\n'
with open(os.path.join(output_dir, '6MV_fluorescence_tissue_filter.txt'), "w") as text_file:
    print(total_str, file=text_file)

## Cherenkov at surface

In [ ]:
cherenkov_at_surface = readMultiGamosOutputFiles(os.path.join(output_dir, 'simulations', root_path_name),
                                          'pdH_Detect_Cerenkov_at_surface')

In [ ]:
surface_ch_0 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='0']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# surface_ch_1 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='1']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# surface_ch_2 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='2']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# surface_ch_3 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='3']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
surface_ch_5 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='5']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# surface_ch_7 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='7']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# surface_ch_10 = plt.hist(cherenkov_at_surface[cherenkov_at_surface['Depth']=='10']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
plt.show()

mean_surface_ch_hist_freq = numpy.nanmean([surface_ch_0[0], surface_ch_5[0]], axis=0)
#                                            surface_ch_1[0],surface_ch_2[0],
#                                            surface_ch_3[0], surface_ch_5[0], surface_ch_7[0],
#                                            surface_ch_10[0]], axis=0)
max_surface_ch_hist_freq = numpy.nanmax(mean_surface_ch_hist_freq )

In [ ]:
surf_ch_df = pandas.DataFrame()
surf_ch_df['wavelength'] = center
surf_ch_df['photons_0mm'] = surface_ch_0[0]
# surf_ch_df['photons_1mm'] = surface_ch_1[0]
# surf_ch_df['photons_2mm'] = surface_ch_2[0]
# surf_ch_df['photons_3mm'] = surface_ch_3[0]
surf_ch_df['photons_5mm'] = surface_ch_5[0]
# surf_ch_df['photons_7mm'] = surface_ch_7[0]
# surf_ch_df['photons_10mm'] = surface_ch_10[0]

# Write fluoresence emission starting info
writer = pandas.ExcelWriter(os.path.join(output_dir, 'Cherenkov_emissions.xlsx'))
surf_ch_df.to_excel(writer, 'Surface')
writer.save()

## Cherenkov exiting tumor

In [ ]:
cherenkov_exit_tumor = readMultiGamosOutputFiles(os.path.join(output_dir, 'simulations', root_path_name),
                                          'pdH_Detect_Cerenkov_exit_tumor')

In [ ]:
exit_ch_0 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='0']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# exit_ch_1 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='1']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# exit_ch_2 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='2']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# exit_ch_3 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='3']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
exit_ch_5 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='5']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
# exit_ch_7 = plt.hist(cherenkov_exit_tumor[cherenkov_exit_tumor['Depth']=='7']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
plt.show()

mean_exit_ch_hist_freq = numpy.nanmean([exit_ch_0[0], exit_ch_5[0]],axis=0)
#                                         exit_ch_1[0],exit_ch_2[0],
#                                         exit_ch_3[0], exit_ch_5[0], exit_ch_7[0],
#                                         exit_ch_10[0]], axis=0)
max_exit_ch_hist_freq = numpy.nanmax(mean_exit_ch_hist_freq )


In [ ]:
exit_ch_df = pandas.DataFrame()
exit_ch_df['wavelength'] = center
exit_ch_df['photons_0mm'] = exit_ch_0[0]
# exit_ch_df['photons_1mm'] = exit_ch_1[0]
# exit_ch_df['photons_2mm'] = exit_ch_2[0]
# exit_ch_df['photons_3mm'] = exit_ch_3[0]
exit_ch_df['photons_5mm'] = exit_ch_5[0]
# exit_ch_df['photons_7mm'] = exit_ch_7[0]
# exit_ch_df['photons_10mm'] = exit_ch_10[0]

# Write fluoresence emission starting info
# writer = pandas.ExcelWriter(os.path.join(output_dir, 'Cherenkov_emissions.xlsx'))
exit_ch_df.to_excel(writer, 'Exit')
writer.save()

## Cherenkov in tumor

In [ ]:
all_ch_df = pandas.DataFrame()

for depth in [0,1,2,3,5,7,10]:
    for i in range(0,10):
        random_seed = 1000+i
        cur_suffix = '{}_{}_{}'.format(root_path_name, depth, random_seed)
#         print(cur_suffix)
        cur_dir = os.path.join(output_dir, 'simulations', root_path_name, cur_suffix)
        tmp_ch_df = readGamosOutputFile(cur_dir,
                            'pdH_Detect_Cerenkov_in_tumor',
                            cur_suffix)
        grouped_df = tmp_ch_df.groupby(['EventID', 'TrackID'], sort=False, as_index=False)["WavelengthEnergy*nm"].first()
        
        grouped_df['Depth'] = depth
    all_ch_df = all_ch_df.append(grouped_df)

    
    

In [ ]:
ch_in_tumor_0 = plt.hist(all_ch_df[all_ch_df['Depth']=='0']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
ch_in_tumor_1 = plt.hist(all_ch_df[all_ch_df['Depth']=='1']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
ch_in_tumor_2 = plt.hist(all_ch_df[all_ch_df['Depth']=='2']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
ch_in_tumor_3 = plt.hist(all_ch_df[all_ch_df['Depth']=='3']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
ch_in_tumor_5 = plt.hist(all_ch_df[all_ch_df['Depth']=='5']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))
ch_in_tumor_7 = plt.hist(all_ch_df[all_ch_df['Depth']=='7']['WavelengthEnergy*nm'],numpy.linspace(350,900, 111))


In [ ]:
in_tumor_ch_df = pandas.DataFrame()
in_tumor_ch_df['wavelength'] = center
in_tumor_ch_df['photons_0mm'] = ch_in_tumor_0[0]
in_tumor_ch_df['photons_1mm'] = ch_in_tumor_1[0]
in_tumor_ch_df['photons_2mm'] = ch_in_tumor_2[0]
in_tumor_ch_df['photons_3mm'] = ch_in_tumor_3[0]
in_tumor_ch_df['photons_5mm'] = ch_in_tumor_5[0]
in_tumor_ch_df['photons_7mm'] = ch_in_tumor_7[0]
in_tumor_ch_df['photons_10mm'] = ch_in_tumor_10[0]

# Write fluoresence emission starting info
# writer = pandas.ExcelWriter(os.path.join(output_dir, 'Cherenkov_emissions.xlsx'))
in_tumor_ch_df.to_excel(writer, 'In_tumor')
writer.save()

## Fluorescence excitation

In [ ]:
for cur_suffix in s3_paths:
    print('{}'.format(cur_suffix))
    print('\tReading input files...')
    secondaries_potential_exc = readGamosOutputFile(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix),
                                      'pdH_Detect_fl_sec_excitation_in_tumor',
                                      cur_suffix)

    fl_start = readGamosOutputFile(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix),
                                      'pdH_Detect_fl_start_tumor',
                                      cur_suffix)
    matching_events = secondaries_potential_exc[secondaries_potential_exc['EventID'].isin(fl_start['EventID'])]
    # Parse out excitation and emission info
    unique_events = fl_start['EventID'].unique()
    matching_tracks_df = pandas.DataFrame()
    valid_fl_events_df = pandas.DataFrame()
    print('\tParsing emission and excitation...')
    for each_event in unique_events:
        # Analyze one event
        cur_fl_slice = fl_start[fl_start['EventID']==each_event]
        # Find tracks that are not sequential
        valid_tracks = numpy.insert(True, 1, numpy.diff(cur_fl_slice['TrackID'])>1)
        cur_valid_tracks = cur_fl_slice[valid_tracks]
        # Find exciation event based on track number
        tmp_df = matching_events[(matching_events['EventID']==each_event) & 
                                 (matching_events['TrackID']+1).isin(cur_valid_tracks['TrackID'])]
        # Append to data frame
        matching_tracks_df = matching_tracks_df.append(tmp_df)
        valid_fl_events_df = valid_fl_events_df.append(cur_valid_tracks)

    print('\tWriting outputs...')
    # Write fluoresence emission starting info
    writer = pandas.ExcelWriter(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix,
                                             'pandas_fl_emission_start_tumor_{}.xlsx'.format(cur_suffix)))
    valid_fl_events_df.to_excel(writer, '{}'.format(cur_suffix))
    writer.save()

    # Write fluoresence excitation info
    writer = pandas.ExcelWriter(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix,
                                             'pandas_fl_excitation_start_tumor_{}.xlsx'.format(cur_suffix)))
    matching_tracks_df.to_excel(writer, '{}'.format(cur_suffix))
    writer.save()

### Read all Excel excitation data

In [ ]:
all_excitation_df = pandas.DataFrame()
fl_emission_df = pandas.DataFrame()



for depth in [0,1,2,3,5,7,10]:
    for i in range(0,10):
        random_seed = 1000+i
        cur_suffix = '{}_{}_{}'.format(root_path_name, depth, random_seed)
        print(cur_suffix)
        tmp_exc_df = pandas.read_excel(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix,
                                                 'pandas_fl_excitation_start_tumor_{}.xlsx'.format(cur_suffix)))
        tmp_exc_df['Depth'] = depth
        all_excitation_df = all_excitation_df.append(tmp_exc_df)
        
        tmp_ems_df = pandas.read_excel(os.path.join(output_dir, 'simulations', root_path_name, cur_suffix,
                                                 'pandas_fl_emission_start_tumor_{}.xlsx'.format(cur_suffix)))
        tmp_ems_df['Depth'] = depth
        fl_emission_df = fl_emission_df.append(tmp_ems_df)
    
    

In [ ]:
fl_exc_0 = plt.hist(all_excitation_df[all_excitation_df['Depth']==0]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_1 = plt.hist(all_excitation_df[all_excitation_df['Depth']==1]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_2 = plt.hist(all_excitation_df[all_excitation_df['Depth']==2]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_3 = plt.hist(all_excitation_df[all_excitation_df['Depth']==3]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_5 = plt.hist(all_excitation_df[all_excitation_df['Depth']==5]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_7 = plt.hist(all_excitation_df[all_excitation_df['Depth']==7]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
fl_exc_10 = plt.hist(all_excitation_df[all_excitation_df['Depth']==10]['WavelengthEnergy*nm'],numpy.linspace(350,700, 71))
width = 0.9 * 5
center = (new_depth_1[1][:-1] + new_depth_1[1][1:]) / 2

In [ ]:
excite_ch_df = pandas.DataFrame()
excite_ch_df['wavelength'] = center
excite_ch_df['photons_0mm'] = fl_exc_0[0]
excite_ch_df['photons_1mm'] = fl_exc_1[0]
excite_ch_df['photons_2mm'] = fl_exc_2[0]
excite_ch_df['photons_3mm'] = fl_exc_3[0]
excite_ch_df['photons_5mm'] = fl_exc_5[0]
excite_ch_df['photons_7mm'] = fl_exc_7[0]
excite_ch_df['photons_10mm'] = fl_exc_10[0]

# Write fluoresence emission starting info
# writer = pandas.ExcelWriter(os.path.join(output_dir, 'Cherenkov_emissions.xlsx'))
excite_ch_df.to_excel(writer, 'Excitation')
writer.save()

## Relative Cherenkov wavelength distribution

In [ ]:
center_all = (ch_in_tumor_0[1][:-1] + ch_in_tumor_0[1][1:]) / 2
bin_width_all = ch_in_tumor_0[1][1]-ch_in_tumor_0[1][0]
center_fl = (fl_exc_1[1][:-1] + fl_exc_1[1][1:]) / 2
bin_width_fl = fl_exc_1[1][1] - fl_exc_1[1][0]
window=5

plt.figure(figsize=[12,12])
in_scale = integrate_spectrum(ch_in_tumor_2[0], center_all, 350,900)
smooth_in = moving_average(100*ch_in_tumor_2[0]/in_scale,window)
plt.plot(moving_average(center_all,window), smooth_in, label='In', lw=6, c='royalblue')


exit_scale = integrate_spectrum(exit_ch_2[0], center_all, 350,900)
smooth_exit = moving_average(100*exit_ch_2[0]/exit_scale,window)
plt.plot(moving_average(center_all,window), smooth_exit, label='Exit', lw=6, c='darkgoldenrod')

surface_scale = integrate_spectrum(surface_ch_2[0], center_all, 350,900)
smooth_surface = moving_average(100*surface_ch_2[0]/surface_scale,window)
plt.plot(moving_average(center_all,window), smooth_surface, label='Surface', lw=6, c='crimson')

fl_exc_scale = integrate_spectrum(new_depth_2[0], center_fl, 350,700)
smooth_exc = moving_average(100*new_depth_2[0]/fl_exc_scale,window)
plt.plot(moving_average(center_fl,window), smooth_exc, label='Excitation', lw=6, ls=':', c='midnightblue')
plt.legend(bbox_to_anchor=(0.5,-0.33), loc="lower center", ncol=2)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Normalized Probability (%)')
plt.xlim(360,890)
plt.ylim(0,0.6)
plt.grid()
plt.savefig(os.path.join(output_dir, 'Cherenkov_probabilities_2mm.png'), bbox_inches='tight')
plt.show()



# Tissue Optical Properties

In [ ]:
to_file = os.path.join(input_dir, 'PtG4_MC_geometry_LaRochelle.xlsx')
xl = pandas.ExcelFile(to_file)
all_layers = {}
layer_names = ['skin1', 'skin2', 'skin3',
               'skin4', 'skin5', 'skin6',
               'skin7', 'fat', 'muscle', 'tumor']
for each_layer in layer_names:
    if each_layer=='tumor':
        all_layers[each_layer] = xl.parse(each_layer, skiprows=8)
    else:
        all_layers[each_layer] = xl.parse(each_layer, skiprows=6)

In [ ]:
attribute_of_interest = 'mu_eff' # Options: 'mu_eff' | 'mu_s_prime' | 'mu_a' | 'n' | 'g'

all_styles = ['-', '--', '-.', ':']
combined_layers = ['epidermis', 'dermis', 'blood-net', 'adipose', 'muscle', 'tumor']
layer_map = [[0,1], [2,4], [3,5], [6,7], [8], [9]]
# define the colormap
cmap = plt.cm.inferno
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_layers = len(combined_layers)
layer_colors = cmap_array[0::int(cmap.N/num_layers)]
color_cycler = cycle(layer_colors)
style_cycler = cycle(all_styles)
              

plt_yy = {}
fig = plt.figure(figsize=[20,15])
ax = fig.gca()
n = 0
for each_layer in combined_layers:
    cur_color = next(color_cycler)
    cur_style = next(style_cycler)
    m=0
    for layer_index in layer_map[n]:
        sub_layer = layer_names[layer_index]
        if sub_layer=='tumor':
            fl_abs = all_layers[sub_layer]['mpt_fl_abs'].values
            fl_ems = all_layers[sub_layer]['mpt_fl_ems'].values
        if m==0:
            cur_wavelengths = all_layers[sub_layer]['mpt_wavelength'].values
            cur_mu_s = all_layers[sub_layer]['mpt_miescat'].values
            cur_mu_a = all_layers[sub_layer]['mpt_abs'].values
            cur_n = all_layers[sub_layer]['mpt_rindex'].values
            cur_g = all_layers[sub_layer]['mpt_g'].values
        else:
            new_wv = all_layers[sub_layer]['mpt_wavelength'].values
            cur_wavelengths = numpy.nanmean([cur_wavelengths,new_wv], axis=0)
            new_mu_s = all_layers[sub_layer]['mpt_miescat'].values
            cur_mu_s = numpy.nanmean([cur_mu_s, new_mu_s], axis=0)
            new_mu_a = all_layers[sub_layer]['mpt_abs'].values
            cur_mu_a = numpy.nanmean([cur_mu_a, new_mu_a], axis=0)
            new_n = all_layers[sub_layer]['mpt_rindex'].values
            cur_n = numpy.nanmean([cur_n, new_n],axis=0)
            new_g = all_layers[sub_layer]['mpt_g'].values
            cur_g = numpy.nanmean([cur_g,new_g], axis=0)
            
        m+=1

    f_cubic_mu_a = interp1d(cur_wavelengths, cur_mu_a, kind='cubic')
    f_cubic_mu_s = interp1d(cur_wavelengths, cur_mu_s, kind='cubic')
    f_n = interp1d(cur_wavelengths, cur_n, kind='cubic')
    f_g = interp1d(cur_wavelengths, cur_g, kind='cubic')
    xx = numpy.linspace(cur_wavelengths.min(),cur_wavelengths.max(),550)
    if len(layer_map[n])>1:
        cur_label = r'$\overline{%s}$'% each_layer
    else:
        cur_label = '${}$'.format(each_layer)
    mu_eff = numpy.sqrt(3*f_cubic_mu_a(xx)*(f_cubic_mu_a(xx)+f_cubic_mu_s(xx)*(1-f_g(xx))))
    plt_yy['mu_eff'] = mu_eff
    plt_yy['mu_s_prime'] = f_cubic_mu_s(xx)*(1-f_g(xx))
    plt_yy['mu_a'] = f_cubic_mu_a(xx)
    plt_yy['n'] = f_n(xx)
    plt_yy['g'] = f_g(xx)
    if (attribute_of_interest=='n') or (attribute_of_interest=='g'):
        plt.plot(xx, plt_yy[attribute_of_interest], color=cur_color, lw=10, label=cur_label, alpha=0.8, ls=cur_style)
    else:
        plt.semilogy(xx, plt_yy[attribute_of_interest], color=cur_color, lw=10, label=cur_label, alpha=0.8, ls=cur_style)

    n+=1
#plt.ylim(0.01, 50)
plt.xlim(350, 900)
plt.grid(True, axis='y', which='major', color='black', linestyle='-', lw=2, alpha=0.5)
plt.grid(True, axis='y', which='minor', color='silver', linestyle=':', lw=1, alpha=0.5)
if attribute_of_interest=='mu_eff':
    plt.ylabel("$\mu_{eff}$ (mm$^{-1}$)", fontsize=55)
elif attribute_of_interest=='mu_s_prime':
    plt.ylabel("$\mu_s^{\'}$ (mm$^{-1}$)", fontsize=55)
elif attribute_of_interest=='mu_a':
    plt.ylabel("$\mu_a$ (mm$^{-1}$)", fontsize=55)
elif attribute_of_interest=='n':
    plt.ylabel('n', fontsize=55)
elif attribute_of_interest=='g':
 plt.ylabel('g', fontsize=55)

ax.yaxis.set_tick_params(labelsize=40)

plt.xlabel('Wavelength (nm)', fontsize=55)
ax.xaxis.set_tick_params(labelsize=40, rotation=-45)

plt.legend(loc='best', fontsize=45)
plt.savefig(os.path.join(output_dir, 'All_{}.png'.format(attribute_of_interest)),bbox_inches='tight')
plt.show()



In [ ]:
fig = plt.figure(figsize=[20,15])
ax = fig.gca()
f_fl_abs = interp1d(cur_wavelengths, fl_abs, kind='cubic')
f_fl_ems = interp1d(cur_wavelengths, fl_ems, kind='cubic')
plt.semilogy(xx,f_fl_abs(xx), color='indigo', lw=10, label='Fluorescence absorption', alpha=0.8, ls='--')
#plt.semilogy(cur_wavelengths, fl_abs,color='indigo', lw=10, label='Fluorescence absorption', alpha=0.8, ls='--')
plt.xlim(350, 900)
plt.grid(True, axis='y', which='major', color='black', linestyle='-', lw=2, alpha=0.5)
plt.grid(True, axis='y', which='minor', color='silver', linestyle=':', lw=1, alpha=0.5)
plt.ylabel("Absorption (mm$^{-1}$)", fontsize=55, color='indigo')
plt.ylim([0.01,1])
ax.yaxis.set_tick_params(labelsize=40, color='indigo')
plt.xlabel('Wavelength (nm)', fontsize=55)
ax.xaxis.set_tick_params(labelsize=40, rotation=-45)

ax2 = ax.twinx()
ax2.plot(xx,f_fl_ems(xx), color='firebrick', lw=10, label='Fluorescence absorption', alpha=0.8, ls=':')
ax2.yaxis.set_tick_params(labelsize=40, color='firebrick')
ax2.set_ylim([0,1])
ax2.set_ylabel("Normalized Emission", fontsize=55, color='firebrick')
plt.savefig(os.path.join(output_dir, 'PtG4_equiv_spectra_50uM.png'), bbox_inches='tight')

plt.show()
# plt.legend(loc='best', fontsize=45)